# --- Day 5: Cafeteria ---

The Elves in the kitchen explain the situation: because of their complicated new inventory management system, they can't figure out which of their ingredients are fresh and which are spoiled. When you ask how it works, they give you a copy of their database (your puzzle input).

The database operates on ingredient IDs. It consists of a list of fresh ingredient ID ranges, a blank line, and a list of available ingredient IDs. For example:
```
3-5
10-14
16-20
12-18

1
5
8
11
17
32
```
The fresh ID ranges are inclusive: the range 3-5 means that ingredient IDs 3, 4, and 5 are all fresh. The ranges can also overlap; an ingredient ID is fresh if it is in any range.

The Elves are trying to determine which of the available ingredient IDs are fresh. In this example, this is done as follows:

- Ingredient ID 1 is spoiled because it does not fall into any range.
- Ingredient ID 5 is fresh because it falls into range 3-5.
- Ingredient ID 8 is spoiled.
- Ingredient ID 11 is fresh because it falls into range 10-14.
- Ingredient ID 17 is fresh because it falls into range 16-20 as well as range 12-18.
- Ingredient ID 32 is spoiled.

So, in this example, 3 of the available ingredient IDs are fresh.

Process the database file from the new inventory management system. How many of the available ingredient IDs are fresh?

In [ ]:
from fastcore.utils import L
from aocd import get_data

inp = get_data(day=5, year=2025)
samp = '''3-5
10-14
16-20
12-18

1
5
8
11
17
32'''
samp

'3-5\n10-14\n16-20\n12-18\n\n1\n5\n8\n11\n17\n32'

Summarize the task for me.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You need to determine how many available ingredient IDs are fresh based on given ranges.

The task breaks down into:

1. **Parse the input** into two parts (separated by a blank line):
   - Fresh ID ranges (e.g., "3-5" means IDs 3, 4, and 5 are fresh)
   - Available ingredient IDs to check

2. **Check each available ID** to see if it falls within any of the fresh ranges

3. **Count** how many of the available IDs are fresh

In the example, out of 6 available IDs (1, 5, 8, 11, 17, 32), only 3 are fresh (5, 11, and 17).

How would you like to approach solving this?

In [ ]:
id_s, ing_s = samp.split("\n\n")
id_s, ing_s

('3-5\n10-14\n16-20\n12-18', '1\n5\n8\n11\n17\n32')

In [ ]:
def parse_rng(s):
    lo, hi = s.split('-')
    return (int(lo), int(hi))
parse_rng('10-14')

(10, 14)

In [ ]:
def parse(s):
    id_str, ing_str = s.split("\n\n")
    ings = L(id_str.splitlines()).map(parse_rng)
    ids = L(ing_str.splitlines()).map(int)
    return ings, ids
rs, ids = parse(samp)
rs, ids

([(3, 5), (10, 14), (16, 20), (12, 18)], [1, 5, 8, 11, 17, 32])

In [ ]:
def in_range(rng, n): return rng[0] <= n <= rng[1]
in_range((3,5), 5)

True

In [ ]:
def in_any_range(n, rs):
    for r in rs:
        if in_range(r, n):
            return True
    return False
in_any_range(5, rs), in_any_range(100, rs)

(True, False)

Can `in_any_range` be implemented in a more functional-style way?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Yes! Since you're working with the `L` class (which appears to be a functional list wrapper), you could use `any()` with a generator expression or map:

```python
def in_any_range(rs, n):
    return any(in_range(r, n) for r in rs)
```

This returns `True` if any range contains `n`, and `False` otherwise. The `any()` function short-circuits, so it stops checking as soon as it finds a match.

Would you like to try implementing this version?

In [ ]:
def in_any_range(n, rs): return any(in_range(r, n) for r in rs)
in_any_range(5, rs), in_any_range(100, rs)

(True, False)

In [ ]:
L(ids).filter(in_any_range, rs=rs)

[5, 11, 17]

In [ ]:
len(L(ids).filter(in_any_range, rs=rs))

3

In [ ]:
def solve(inp):
    rs, ids = parse(inp)
    return len(L(ids).filter(in_any_range, rs=rs))
solve(samp)

3

In [ ]:
solve(inp)

821

# --- Part Two ---
The Elves start bringing their spoiled inventory to the trash chute at the back of the kitchen.

So that they can stop bugging you when they get new inventory, the Elves would like to know all of the IDs that the fresh ingredient ID ranges consider to be fresh. An ingredient ID is still considered fresh if it is in any range.

Now, the second section of the database (the available ingredient IDs) is irrelevant. Here are the fresh ingredient ID ranges from the above example:
```
3-5
10-14
16-20
12-18
```
The ingredient IDs that these ranges consider to be fresh are 3, 4, 5, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, and 20. So, in this example, the fresh ingredient ID ranges consider a total of 14 ingredient IDs to be fresh.

Process the database file again. How many ingredient IDs are considered to be fresh according to the fresh ingredient ID ranges?

Is there some useful structure to represent interval of numbers? I'll need an operation to merge the intervals.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Yes! You'll want to work with intervals as tuples `(start, end)` and merge overlapping or adjacent ones.

The key insight is that if you sort the intervals by their start position, you can merge them in one pass. Two intervals can be merged if they overlap or touch (e.g., `(3, 5)` and `(5, 7)` can merge into `(3, 7)`).

The merging condition is: if the start of the next interval is ≤ the end of the current merged interval (plus 1 for adjacency), they can merge.

Would you like to try implementing a function that takes a list of intervals and returns the merged intervals?

Ok, that's useful! But isn't there a library for this?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good thinking! Yes, there are libraries that handle intervals. The `portion` library is popular for interval operations in Python, including unions and intersections.

However, for Advent of Code problems like this, merging intervals manually is usually straightforward enough and avoids adding dependencies. Plus, it's a good algorithmic exercise!

Do you want to implement the merge yourself, or would you prefer to explore using a library like `portion`?

The ranges before sorting:
```
3-5
10-14
16-20
12-18
```
After sorting:
```
3-5
10-14
12-18
16-20
```

Ok, let's implement it. I'll need to be able to remove one or two ranges and replace them with a new range. Is there a sorted list datastructure?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You could use Python's `bisect` module with a regular list to maintain sorted order, or the `sortedcontainers` library which provides `SortedList`.

But for merging intervals, you don't actually need to maintain a sorted structure during the merge! The typical approach is simpler:

1. Sort all intervals once at the start
2. Iterate through them, building up a result list of merged intervals
3. For each interval, either merge it with the last interval in your result, or append it as a new interval

This way you're just appending to a regular list, not inserting/removing from a sorted structure. Would you like to try implementing it this way?

There are these cases:
1. The currently iterated interval completely fits into the last interval. Then do nothing.
2. The currently iterated interval doesn't overlap with the last interval. Then append.
3. The currently iterated interval has an overlap but its end value is greater than the end value of the last interval. Then merge - adjust the end value of the last interval.

Does the reasoning look correct?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Yes, your reasoning is correct! You've identified all the cases needed for merging sorted intervals.

One small note: cases 1 and 3 can actually be handled together by the same logic. When there's overlap (or the current interval fits completely inside), you can just update the end of the last interval to be `max(last_end, current_end)`. This covers both situations.

So you really have two cases:
- **No overlap**: append the current interval as new
- **Overlap or contained**: update the end of the last interval to the maximum of the two ends

Does that simplification make sense?

In [ ]:
def acc_fn(acc, curr):
    if not acc: return [curr]
    curr_lo, curr_hi = curr
    lst_lo, lst_hi = lst_rng = acc[-1]
    if curr_lo <= lst_hi:
        acc[-1] = (lst_lo, max(lst_hi, curr_hi))
    else:
        acc.append(curr)
    return acc
acc_fn([(10,14)], (12,18))

[(10, 18)]

In [ ]:
from functools import *


In [ ]:
reduce?

Docstring:
reduce(function, iterable[, initial]) -> value

Apply a function of two arguments cumulatively to the items of an iterable, from left to right.

This effectively reduces the iterable to a single value.  If initial is present,
it is placed before the items of the iterable in the calculation, and serves as
a default when the iterable is empty.

For example, reduce(lambda x, y: x+y, [1, 2, 3, 4, 5])
calculates ((((1 + 2) + 3) + 4) + 5).
Type:      builtin_function_or_method

In [ ]:
rs = sorted(rs)
rs

[(3, 5), (10, 14), (12, 18), (16, 20)]

In [ ]:
xs = reduce(acc_fn, rs, [])
xs

[(3, 5), (10, 20)]

In [ ]:
def cnt(rng): return rng[1] - rng[0] + 1
cnt((3, 5))


3

In [ ]:
L(xs).map(cnt)

[3, 11]

In [ ]:
L(xs).map(cnt).sum()

14

In [ ]:
def solve(inp):
    rs, _ = parse(inp)
    rs = sorted(rs)
    xs = reduce(acc_fn, rs, [])
    return L(xs).map(cnt).sum()
solve(samp)

14

In [ ]:
solve(inp)

344771884978261